In [1]:
import pandas as pd
import time
from datetime import datetime

# Placeholder function to get the current market price; replace with actual implementation
def get_current_market_price(ticker):
    # Implement the logic to fetch the current market price for the given ticker
    # This is a mock implementation; replace it with actual market data fetch
    return 100  # Example price, replace with actual market price

# Initialize the trade details
trade_details = {
    'Date': entry_date,
    'Expiry': expiry_date,
    'En_Date': entry_date,
    'Pe_En_Date': entry_date,
    'Ce_Ex_Date': None,
    'Pe_Ex_Date': None,
    'Atm_Strike': atm_strike_price,
    'Pe_Short_Strike': put_options_atm['Strike Price'],
    'Pe_otm_Long_Strike': put_options_otm['Strike Price'],
    'Pe_itm_Long_Strike': put_options_itm['Strike Price'],
    'Capital': capital,
    'Pe_Short_En_Price': put_options_atm['Close'],
    'Pe_Short_Ex_Price': None,
    'Pe_otm_Long_En_Price': put_options_otm['Close'],
    'Pe_otm_Long_Ex_Price': None,
    'Pe_itm_Long_En_Price': put_options_itm['Close'],
    'Pe_itm_Long_Ex_Price': None,
    'Entry_Time': entry_time,
    'Exit Time': None,
    'Current Market Price': None,
    'Ticker_Pe_Short': put_options_atm['Ticker'],
    'Ticker_otm_Pe_Long': put_options_otm['Ticker'],
    'Ticker_itm_Pe_Long': put_options_itm['Ticker'],
    'Exit_Reason': None,
    'Stop_Loss': stop_loss,
    'Total_Premium_Received': total_entry_premium,
    'Target': target,
    'Total_PnL': None
}

# Initialize an empty list to store P&L data
pnl_data = []

# Function to calculate P&L for the trade
def calculate_pnl(trade_details):
    pe_short_current_price = get_current_market_price(trade_details['Ticker_Pe_Short'])
    pe_otm_long_current_price = get_current_market_price(trade_details['Ticker_otm_Pe_Long'])
    pe_itm_long_current_price = get_current_market_price(trade_details['Ticker_itm_Pe_Long'])
    
    pe_short_pnl = trade_details['Pe_Short_En_Price'] - pe_short_current_price
    pe_otm_long_pnl = pe_otm_long_current_price - trade_details['Pe_otm_Long_En_Price']
    pe_itm_long_pnl = pe_itm_long_current_price - trade_details['Pe_itm_Long_En_Price']
    
    total_pnl = pe_short_pnl + pe_otm_long_pnl + pe_itm_long_pnl
    return total_pnl

# Main loop to check P&L every minute
while True:
    trade_details['Current Market Price'] = {
        'Pe_Short': get_current_market_price(trade_details['Ticker_Pe_Short']),
        'Pe_otm_Long': get_current_market_price(trade_details['Ticker_otm_Pe_Long']),
        'Pe_itm_Long': get_current_market_price(trade_details['Ticker_itm_Pe_Long']),
    }
    
    trade_details['Total_PnL'] = calculate_pnl(trade_details)
    pnl_data.append({
        'Time': datetime.now(),
        'Total_PnL': trade_details['Total_PnL']
    })
    
    print(f"{datetime.now()}: P&L - {trade_details['Total_PnL']}")
    
    # Check if target or stop-loss is hit
    if trade_details['Total_PnL'] >= trade_details['Target']:
        trade_details['Exit_Reason'] = 'Target Hit'
        break
    elif trade_details['Total_PnL'] <= trade_details['Stop_Loss']:
        trade_details['Exit_Reason'] = 'Stop Loss Hit'
        break
    
    time.sleep(60)  # Wait for 1 minute

# Save and display the final result
trade_details['Exit Time'] = datetime.now()

pnl_df = pd.DataFrame(pnl_data)
pnl_df.to_csv('pnl_data.csv', index=False)
print("P&L data saved to pnl_data.csv")

# Display the DataFrame to the user
import ace_tools as tools
tools.display_dataframe_to_user(name="P&L Data", dataframe=pnl_df)
